In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Dropout, Activation,Flatten,Conv2D,MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm

In [ ]:
data = pd.read_csv('data/semeion.data', delimiter='\s+')

In [ ]:
def label_transform(df):
    ls= df.values
    label=0
    for i in ls:
        if i==1:
            break
        elif i==0:
            label+=1
        else:
            print("wrong format")
    return label

In [ ]:
def data_transform(df):
    vl = df.values
    data = np.array( vl.reshape((16,16)))
    return data

In [ ]:
data_label = data.iloc[:,256:266]

In [ ]:
label_list = []
data_list = []
for i in range(len(data_label.index)):
    label = label_transform(data_label.loc[i])
    label_list.append(label)
for u in range(len(data.index)):
    data_i = data_transform(data.loc[u].drop(data_label.columns))
    data_list.append(data_i)
data_list = np.array(data_list)
label_list = np.array(label_list)

In [ ]:
data_list=  data_list.reshape(data_list.shape[0], 16, 16,1)

In [ ]:
#Split the data
X_train, X_test, y_train, y_test = train_test_split(data_list,label_list, test_size=0.2)

In [ ]:
# custom activation
from keras.backend import sigmoid
def swish(x, beta = 1.52):
    return (x * sigmoid(beta * x))
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
# Weight Constraints
norm = max_norm(2.7)

In [22]:
#Simulate diffrent combination layers and parammeter
import time
histories=[]
modelss = []
denses = [1,2,3]
cn_layers = [0,1,2,3]
layer_sizes = [64,128,256,512]
stack_cns = [1,2,3]
filter_sizes = [16,32]
drop_out_rates = [0,0.3,0.4,0.5]
earlystopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, mode='auto')


for drop_out_rate in drop_out_rates:
    model = Sequential()
    model.add(Dense(256,input_shape=(16, 16, 1) ,kernel_constraint=norm, bias_constraint=norm))
    model.add(Flatten())
    model.add(Dense(512, activation='swish' ,kernel_initializer="glorot_normal"))
    model.add(Dropout(drop_out_rate))
    model.add(Dense(256, activation='swish' ,kernel_initializer="glorot_normal"))
    model.add(Dropout(drop_out_rate))
    model.add(Dense(10,activation='softmax',kernel_initializer='uniform'))
    adam = Adam(learning_rate=0.00092, amsgrad=True)
    model.compile( optimizer = adam, loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs = 2000,validation_split = 0.2,shuffle = True, verbose=0,batch_size=4096
                                    , callbacks = [earlystopper]
                                   )
    model.summary() 
    test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
    history_dict=history.history
    acc_values = max(history_dict['accuracy'])
    print("Train accuracy",acc_values)
    val_acc_values=max(history_dict['val_accuracy'])
    print("Validation accuracy",val_acc_values)
    print('\nTest accuracy:', test_acc)
    print("Drop out rate:",drop_out_rate)
    modelss.append(model)
    histories.append(history)
for stack_cn in stack_cns:
      for filter_size in filter_sizes:
            for drop_out_rate in drop_out_rates:
                  for cn_layer in cn_layers:
                        if cn_layer==1:
                            model = Sequential()
                            model.add(Conv2D(filter_size, (3, 3),
                                 input_shape=(16, 16, 1) ,kernel_constraint=norm, bias_constraint=norm))
                            model.add(MaxPooling2D((2, 2)))
                            model.add(Dropout(drop_out_rate))
                            model.add(Flatten())
                            model.add(Dense(512, activation='swish' ,kernel_initializer="glorot_normal"))
                            model.add(Dropout(drop_out_rate))
                            model.add(Dense(256, activation='swish' ,kernel_initializer="glorot_normal"))
                            model.add(Dropout(drop_out_rate))
                            model.add(Dense(10,activation='softmax',kernel_initializer='uniform'))
                            adam = Adam(learning_rate=0.00092, amsgrad=True)
                            model.compile( optimizer = adam, loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
                            history =model.fit(X_train, y_train, epochs = 2000,validation_split = 0.2,shuffle = True, verbose=0,batch_size=4096
                                    , callbacks = [earlystopper]
                                   )
                            model.summary()
                            test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
                            history_dict=history.history
                            acc_values = max(history_dict['accuracy'])
                            print("Train accuracy",acc_values)
                            val_acc_values=max(history_dict['val_accuracy'])
                            print("Validation accuracy",val_acc_values)
                            print("\nTest accuracy:", test_acc)
                            print("Drop out rate:",drop_out_rate)
                            modelss.append(model)
                            histories.append(history)
                        if cn_layer == 2:
                            model = Sequential()
                            model.add(Conv2D(filter_size, (3, 3), 
                                 input_shape=(16, 16, 1) ,kernel_constraint=norm, bias_constraint=norm))
                            model.add(MaxPooling2D((2, 2)))
                            if stack_cn==1:
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            elif stack_cn==2:
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            elif stack_cn==3:
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            model.add(Dropout(drop_out_rate))
                            model.add(Flatten())
                            model.add(Dense(512, activation='swish' ,kernel_initializer="glorot_normal"))
                            model.add(Dropout(drop_out_rate))
                            model.add(Dense(256, activation='swish' ,kernel_initializer="glorot_normal"))
                            model.add(Dropout(drop_out_rate))
                            model.add(Dense(10,activation='softmax',kernel_initializer='uniform'))
                            adam = Adam(learning_rate=0.00092, amsgrad=True)
                            model.compile( optimizer = adam, loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
                            history = model.fit(X_train, y_train, epochs = 2000,validation_split = 0.2,shuffle = True, verbose=0,batch_size=4096
                                    , callbacks = [earlystopper]
                                   )
                            model.summary()
                            test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
                            history_dict=history.history
                            acc_values = max(history_dict['accuracy'])
                            print("Train accuracy",acc_values)
                            val_acc_values=max(history_dict['val_accuracy'])
                            print("Validation accuracy",val_acc_values)
                            print('\nTest accuracy:', test_acc)
                            print("Drop out rate:",drop_out_rate)
                            modelss.append(model)
                            histories.append(history)
                        if cn_layer == 3:
                            model = Sequential()
                            model.add(Conv2D(filter_size, (3, 3),  
                                 input_shape=(16, 16, 1) ,kernel_constraint=norm, bias_constraint=norm))
                            model.add(MaxPooling2D((2, 2)))
                            if stack_cn==1:
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            elif stack_cn==2:
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            elif stack_cn==3:
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*2, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            if stack_cn==1:
                                model.add(Conv2D(filter_size*4, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            elif stack_cn==2:
                                model.add(Conv2D(filter_size*4, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*4, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            elif stack_cn==3:
                                model.add(Conv2D(filter_size*4, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*4, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(Conv2D(filter_size*4, (3, 3),  padding='same',kernel_constraint=norm, bias_constraint=norm))
                                model.add(MaxPooling2D((2, 2)))
                            model.add(Dropout(drop_out_rate))
                            model.add(Flatten())
                            model.add(Dense(512, activation='swish' ,kernel_initializer="glorot_normal"))
                            model.add(Dropout(drop_out_rate))
                            model.add(Dense(256, activation='swish' ,kernel_initializer="glorot_normal"))
                            model.add(Dropout(drop_out_rate))
                            model.add(Dense(10,activation='softmax',kernel_initializer='uniform'))
                            adam = Adam(learning_rate=0.00092, amsgrad=True)
                            model.compile( optimizer = adam, loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])
                            history = model.fit(X_train, y_train, epochs = 2000,validation_split = 0.2,shuffle = True, verbose=0,batch_size=4096
                                    , callbacks = [earlystopper]
                                   )
                            model.summary()
                            test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
                            history_dict=history.history
                            acc_values = max(history_dict['accuracy'])
                            print("Train accuracy",acc_values)
                            val_acc_values=max(history_dict['val_accuracy'])
                            print("Validation accuracy",val_acc_values)
                            print('\nTest accuracy:', test_acc)
                            print("Drop out rate:",drop_out_rate)
                            modelss.append(model)
                            histories.append(history)

Epoch 00057: early stopping
Model: "sequential_77"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_233 (Dense)            (None, 16, 16, 256)       512       
_________________________________________________________________
flatten_77 (Flatten)         (None, 65536)             0         
_________________________________________________________________
dense_234 (Dense)            (None, 512)               33554944  
_________________________________________________________________
dropout_225 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_235 (Dense)            (None, 256)               131328    
_________________________________________________________________
dropout_226 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_236 (Dense)        